# HW3 Q4 [10 pts]



## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>


All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  



<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`load_data()`

`exclude_no_pickuplocations()`

`exclude_no_tripdistance()`

`include_fare_range()`

`get_highest_tip()`

`get_total_toll()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from NYC Taxi Trips during auto-grading. You do not need to write code for unreasonable scenarios.  

Since the overall correctness of your code will require multiple function to work together correctly (i.e., all methods are interdepedent), implementing only a subset of the functions likely will lead to a low score.

### Helper functions

You are permitted to write additional helper functions, or use additional instance variables so long as the previously described functions work as required.  

#### Pyspark Imports
<span style="color:red">*Please don't modify the below cell*</span>

In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import *

#### Define Spark Context
<span style="color:red">*Please don't modify the below cell*</span>

In [2]:
sc
sqlContext = SQLContext(sc)

### Student Section - Please compete all the functions below

In [3]:
def user():
        """
        :return: string
        """         
        return 'mganesan7'

#### Function to load data

In [4]:
def load_data(gcp_storage_path):
    """
        :param gcp_storage_path: string (full gs path including file name e.g gs://bucket_name/data.csv) 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to load yellow_tripdata_2019-01.csv data from your GCP storage bucket#
    #                                                              #        
    ################################################################
    #gcp_storage_path = 'gs://mganesan7/yellow_tripdata_2019-01.csv'
    from pyspark.sql.types import DecimalType
    from pyspark.sql import functions as F
    df = spark.read.format('csv').option("header", "true").load([gcp_storage_path]) \
              .withColumn("trip_distance",F.col('trip_distance').cast(DecimalType())) \
              .withColumn("fare_amount",F.col('fare_amount').cast(DecimalType())) \
              .withColumn("tip_amount", F.col('tip_amount').cast(DecimalType())) \
              .withColumn("tolls_amount", F.col('tolls_amount').cast(DecimalType()))
        
    return df

#### Function to exclude trips that don't have a pickup location

In [5]:
def exclude_no_pickuplocations(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to exclude trips with no pickup locations               #
    # Note: Exclude  nulls, blanks and zeros                       #        
    ################################################################
    from pyspark.sql import functions as F
    
    df = df.filter(df.PULocationID.isNotNull())
    df = df.filter(df.PULocationID != "")
    df = df.filter(df.PULocationID != 0.0)
    
    return df

#### Function to exclude trips with no distance

In [6]:
def exclude_no_tripdistance(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to exclude trips with no trip distances                 #
    # Note: Exclude  nulls, blanks and zero                        #        
    ################################################################
    from pyspark.sql import functions as F
    df = df.na.drop(subset = ["trip_distance"]).filter(df.trip_distance != 0.0)
    
    return df

#### Function to include fare amount between the range of 20 to 60 Dollars

In [7]:
def include_fare_range(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to include trips with only within the fare range of     #
    # 20 to 60 dollars (including 20 and 60 dollars)               #        
    ################################################################
    
    from pyspark.sql import functions as F
    df = df.filter( (df.fare_amount >= 20) & (df.fare_amount <= 60) )
    
    return df

#### Function to get the highest tip amount

In [8]:
def get_highest_tip(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get the highest tip                                  #
    #                                                              #        
    ################################################################
    from pyspark.sql import functions as F
    from pyspark.sql.types import DecimalType
    
    max_tip = df.groupBy().max('tip_amount') \
     .withColumn("max_tip",F.round(F.col('max(tip_amount)'),2).cast("decimal(10,2)")) \
     .collect()[0]['max_tip']
    
    return max_tip

#### Function to get total toll amount

In [9]:
def get_total_toll(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get total toll amount                                #
    #                                                              #        
    ################################################################
    from pyspark.sql import functions as F
    from pyspark.sql.types import DecimalType
    
    total_toll = df.groupBy().agg({'tolls_amount':'sum'}) \
                   .withColumnRenamed('sum(tolls_amount)','total_toll') \
                   .withColumn('total_toll', F.round(F.col("total_toll"),2).cast("decimal(10,2)")) \
                   .collect()[0]['total_toll']
    
    return total_toll

### Run above functions and print

#### Uncomment the cells below and test your implemented functions

#### Load data from yellow_tripdata_2019-01.csv 

In [10]:
gcp_storage_path = "gs://mganesan7/yellow_tripdata_2019-01.csv"
df = load_data(gcp_storage_path)
df.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: string (nullable = true)
 |-- trip_distance: decimal(10,0) (nullable = true)
 |-- RatecodeID: string (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: decimal(10,0) (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: decimal(10,0) (nullable = true)
 |-- tolls_amount: decimal(10,0) (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



#### Print total numbers of rows in the dataframe

In [11]:
df.count()

7667792

#### Print total number of rows in the dataframe after excluding trips with no pickup location

In [12]:
df_no_pickup_locations = exclude_no_pickuplocations(df)
df_no_pickup_locations.count()

7667792

#### Print total number of rows in the dataframe after exclude trips with no distance

In [13]:
df_no_trip_distance = exclude_no_tripdistance(df)
df_no_trip_distance.count()

7230868

#### Print total number of rows in the dataframe after including trips with fair amount between the range of 20 to 60 Dollars

In [14]:
df_include_fare_range = include_fare_range(df)
df_include_fare_range.count()

1011593

#### Print the highest tip amount

In [15]:
max_tip = get_highest_tip(df)
print(max_tip)

787.00


#### Print the total toll amount

In [16]:
total_toll = get_total_toll(df)
print(total_toll)

2525415.00
